Code
----

In [1]:
# install package
%pip install evaluate
%pip install bert_score
%pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4711 sha256=7deb14260c3146458b94a100babb12de393f002776fb321f3192f7b220a949b4
  Stored in directory: /root/.cache/pip/wheels/c0/96/80/1aeaaf27265398a7393b296714b84e51a7d00101f8a665a25a
Suc

In [2]:
# import necessary packages 
import torch, requests, json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import numpy as np   
import pandas as pd
from tqdm.notebook import tqdm  
tqdm.pandas()
from evaluate import load
import gradio as gr

In [3]:
# prepare tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')
model = AutoModelForQuestionAnswering.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

In [4]:
# method to convert SQuAD-V2 train dataset from json to dataframe
def squad_json_to_dataframe(input_file_path, record_path = ['data','paragraphs','qas','answers'], verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.json_normalize(file , record_path )
    m = pd.json_normalize(file, record_path[:-1] )
    r = pd.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],axis=1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [5]:
# convert SQuAD-V2 train dataset from json to dataframe
input_file_path = '/kaggle/input/squad-2-dataset/train-v2.0.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...
shape of the dataframe is (130319, 6)
Done


In [6]:
train.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0


In [7]:
# data cleanup
train = train.drop(['index','c_id','answer_start'],axis=1)
train.head()

,question,context,text
0,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s
1,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003
3,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas"
4,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s


In [8]:
# method to generate answer for a given question from a given context
def predict_answer(question,context):
  encoded_text = tokenizer.encode_plus(text=question,text_pair=context)
  inputs = encoded_text['input_ids'] 
  sentence_embeddings = encoded_text['token_type_ids']
  tokens = tokenizer.convert_ids_to_tokens(inputs)
  outputs = model(input_ids=torch.tensor([inputs]),token_type_ids=torch.tensor([sentence_embeddings]))
  start_index = torch.argmax(outputs.start_logits).item()
  end_index = torch.argmax(outputs.end_logits).item()
  answer = ' '.join(tokens[start_index:end_index+1])
  corrected_ans = ''

  for word in answer.split():
    if word[:2] == '##':
      corrected_ans += word[2:]
    else:
      corrected_ans += ' ' + word
  return corrected_ans.strip()

In [9]:
question = "Cristiano Ronaldo currently playing at which club?"
context = "Cristiano Ronaldo dos Santos Aveiro (born 5 February 1985), better known as Cristiano Ronaldo, is a Portuguese professional footballer who plays as a forward. He is the captain of the Portuguese national team and he is currently playing at Saudi Arabian football club Al Nassr. He is considered to be one of the greatest footballers of all time, and, by some, as the greatest ever. Ronaldo began his professional career with Sporting CP at age 17 in 2002, and signed for Manchester United a year later. He won three back-to-back Premier League titles: in 2006-07, 2007-08, and 2008-09. In 2007-08, Ronaldo, helped United win the UEFA Champions League. In 2008-09, he won his first FIFA Club World Cup in December 2008, and he also won his first Ballon d'Or. At one point Ronaldo was the most expensive professional footballer of all time, after moving from Manchester United to Real Madrid for approximately £80 m in July 2009."
predict_answer(question,context)

'Al Nassr'

In [10]:
# generate predicted answers from training sample
train_sample = train.sample(n=100).dropna()
predicted_answers = []

for i in tqdm(range(len(train_sample))):
  ques = train_sample.question.iloc[i]
  ans = train_sample.text.iloc[i]
  predicted_answers.append(predict_answer(ques,ans))

  0%|          | 0/65 [00:00<?, ?it/s]

In [11]:
# calculate bert scores (precision, recall, f1, hashcode)
bert_score = load("bertscore")
results = bert_score.compute(predictions=predicted_answers,references=train_sample.text.values,lang='en')

In [12]:
precision, recall, f1_scores, hashcode = results['precision'], results['recall'], results['f1'], results['hashcode']

In [13]:
train_sample['answer'] = predicted_answers
train_sample['f1_score'] = f1_scores
train_sample.head(10)

,question,context,text,answer,f1_score
11385,What's the title of the fifth film in the Term...,"On May 20, 2011, Schwarzenegger's entertainmen...",Terminator Genisys,Terminator Genisys,1.000000
93766,What cost is often regarded as marking the ach...,The PV industry has seen drops in module price...,$1.00/W installed cost,$ 1 . 00 / W installed cost,0.947225
23654,Of what ancestry are most Marshall Islands cit...,The majority of the citizens of the Marshall I...,Marshallese,Marshallese,1.000000
3905,At what location is a Christmas tree famously ...,Major tourist destinations include Times Squar...,Rockefeller Center,Rockefeller Center,1.000000
20130,When did Nanjing become the Chinese national c...,"Nanjing ( listen; Chinese: 南京, ""Southern Capit...",Jin dynasty,Jin dynasty,1.000000
86994,Is there a test to determine asthma in an indi...,There is currently no precise test with the di...,There is currently no precise test,There is currently no precise test,1.000000
1399,What writing of Chopin talks about everything ...,"In 1841, Léon Escudier wrote of a recital give...",his Projet de méthode,Projet de méthode,0.950939
56105,When did the ANSI define its new measure?,Based on earlier research work by Loyd Ancile ...,1943,1943,1.000000
129282,What did hunting become a theme of?,While it is undisputed that early humans were ...,stories and myths,stories and myths,1.000000
4218,What percentage of the New York City cab fleet...,New York City has focused on reducing its envi...,28%,28 %,0.955267


In [14]:
# render an user interface
app = gr.Interface(fn=predict_answer, inputs = ['textbox', 'text'], outputs = 'textbox', title = 'Question Answering bot', description = 'Input context and question, then get answers!')
app.launch(inline=True)

Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on local URL:  http://127.0.0.1:7860
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Running on public URL: https://a1f34caada0bb6680b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Conclusion
----------
Here I have used AutoTokenizer and AutoModelForQuestionAnswering along with cased version of Bert Large pretrained model from Huggingface. In the final result, the model is able to predict the answers for even those questions which are not directly mentioned in the context, but some related statement is present in the context. This way it is different from any normal Text search algorithm.